# Working in AE5 with Multiple Envs
## Some Basic AE5 sessions Concepts
 - Anaconda enterprise launches a pod with 4 containers for each session opened by any user:
   - 2 x containers with ae-editor docker image , one for the actual editor the other for running a git synching service
   - 2 x containers with ae-app-proxy docker image, one for each of the above so communication to/from is using the authentication infrastructure of AE5
 - the ae-editor image, is a huge docker image and it is the current mechanism we adopted to deliver the pre-canned environments to all the cluster node (this will be handled by the container runtime and will be cached on all hosts, if configured with high performance storage, that will provide better performance).
 - the canned environments shipped with 5.4.0 are:
 * noticeably anaconda_50_r is missing and that will be rectified in 4.5.1

## The official environments for 5.4.0 are as follows... ##
 **anaconda201908_py37** - the only python 3.7 environment - this should probably be used for most cases
 anaconda44_(py27,py35,py36 & r)
 anaconda50_(hadoop,impyla,py27,py35,py36,sas)

 - Other environments used by AE5 - not for developing projects
 base, lab_launch & sync_launch

 - base is where conda is installed, - lab_launch has many binaries linked from base (like conda for example)
 - lab_launch is where the editor environment is installed (Jupyter notebook/lab zeppelin, etc...) as well as all the anaconda-project and other supporting tools for preparing the session.
 - environment base is in /opt/continuum/anaconda, all other envs are /opt/continuum/anaconda/envs/<env-name>


 ANACONDA_PROJECT_ENVS_PATH is set so that new environment created by anaconda-project are created in the base folder (which is the docker overlay storage) instead of where the command is executed (avoiding committing the binaries to git)


## Environment Preperation ##
 When a session is launched, the first thing that will happen is the first environment is being prepared. The environments and their pakages are listed in anaconda-project.yml and you can have a many as you like.
 However they will not be prepared (only the first will) this is the most common use of AE5, a single environment and kernel per project.

## Tl;DR ##
 In order to use multiple environment follow the following steps:
 1. add environment spec to anaconda-project.yaml
 2. use anaconda-project prepare to prepare the environments
 3. use the Project tab to update jupyter lab environment (launcher, terminals and kernels)
 4. optional rename and install other kernels

## Details ##
 - You can prepare the environment from the command line with anaconda-project prepare --env-spec and the name of the env
 - or you could update anaconda-project to the latest version and prepare all environments with --all.
 - In either case you have to do that from the terminal and from the lab_launch environment (not from the notebook).
 An easy way to simplify this, is to include a standard set of scripts in your project that can simplify tasks this - like I have here in *scripts/prep.sh*

 ```conda update anaconda-project -y && anaconda-project prepare --all```

 - With multiple environment in anaconda-project.yml and with the environments prepared, you only need to click on any of the preparation ticks in the ***Project tab*** so that kernelspecs will be added and registered in the launcher and as available kernelspec in the notebooks.
 - **Note that this step is mandatory** you will not be able to use the multiple environments without clicking the check-marks. Also, note that the anaconda-project prepare is also mandatory (just ticking the check marks will not prepare the environments).
 - Tip1: If you include envspec in anaconda-project yaml with no packages - as in ```anaconda201908_py37: {}``` (prebuilt env) the startup time will be very fast.
 - Tip2: You can then add packages to the env if they are missing. If you specify the exact version you would like the preparation will be faster.

## Optional Additions ##

- Next you may notice that the launcher does not select meaningful names for the kernels being used, you can rename the kernels to the environment name if you like, it is a string in each kernel.json in each environment.
- I do this with a small function in scripts/prep.sh you can take a look at that to see how that works.
- Now that you have multiple environments, you can also list multiple commands each with the desired environment. see how this is done in anaconda-project.yml
- with this configuration when you deploy a project you can use the desired environment
- I included in this project a Julia kernel that is not part of AE5 supported languages (no conda packages for Julia in AD). You can examine this to understand the flexibility of the platform. the scripts folder includes the needed commands to install Julia environment and the kernel.


In [1]:
!conda env list | awk '!/#/ {print $1}'

base
Julia
anaconda201908_py37
anaconda44_py27
anaconda44_py35
anaconda44_py36
anaconda44_r
anaconda50_hadoop
anaconda50_impyla
anaconda50_py27
anaconda50_py35
anaconda50_py36
anaconda50_sas
lab_launch
py27
py36
py37
sync_launch



In [ ]:
!env | grep ANACONDA_PROJECT_ENVS_PATH

In [ ]:
!conda env list